In [1]:
%autosave 180
%load_ext autoreload
%autoreload 2

Autosaving every 180 seconds


In [2]:
%%capture
#!pip install catboost
!pip install shap

In [25]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
  Using cached pyarrow-8.0.0-cp38-cp38-macosx_10_13_x86_64.whl (22.4 MB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os
import gc

In [48]:
import pickle
import numpy as np
import pandas as pd
from lightgbm import LGBMRanker

# Params

In [19]:
original_cols = [
    'srch_id',
    #     'date_time',  # not used in models
    'site_id',
    #     'visitor_location_country_id',
    'visitor_hist_starrating',
    'visitor_hist_adr_usd',
    'prop_country_id',
    'prop_id',
    'prop_starrating',
    'prop_review_score',
    'prop_brand_bool',
    'prop_location_score1',
    'prop_location_score2',
    'prop_log_historical_price',
    'price_usd',
    'promotion_flag',
    'srch_destination_id',
    #     'srch_length_of_stay',
    'srch_booking_window',
    #     'srch_adults_count',
    'srch_children_count',
    #     'srch_room_count',
    'srch_saturday_night_bool',
    'srch_query_affinity_score',
    'orig_destination_distance',
    'random_bool',
    #     'comp1_inv',
    #     'comp1_rate_percent_diff',
    #     'comp2_inv',
    'comp2_rate_percent_diff',
    #     'comp3_inv',
    'comp3_rate_percent_diff',
    #     'comp4_inv',
    #     'comp4_rate_percent_diff',
    #     'comp5_inv',
    'comp5_rate_percent_diff',
    #     'comp6_inv',
    #     'comp6_rate_percent_diff',
    #     'comp7_inv',
    #     'comp7_rate_percent_diff',
    #     'comp8_inv',
    #     'comp8_rate_percent_diff',
]

booking_date_features = [
    'date_time_ordinal',
    #     'booking_time_minute',
    #     'booking_time_hour',
    #     'booking_time_morning',
    #     'booking_time_afternoon',
    #     'booking_time_evening',
    #     'booking_time_late_evening',
    #     'booking_time_night',
    #     'booking_time_work_hour',
    #     'booking_time_lunch',
    #     'booking_time_day',
    #     'booking_time_is_month_start',
    #     'booking_time_is_month_end',
    #     'booking_time_dow',
    #     'booking_time_is_weekend',
    #     'booking_time_doy',
    #     'booking_time_week',
    #     'booking_time_week_mid_summer_index',
    #     'booking_time_month',
    #     'booking_time_days_in_month',
    #     'booking_time_quarter',
    #     'booking_time_is_quarter_start',
    #     'booking_time_is_quarter_end',
    #     'booking_time_season_num',
    #     'booking_time_year',
    #     'booking_time_is_year_start',
    #     'booking_time_is_year_end',
    #     'booking_time_week_id',
    #     'booking_time_day_sin',
    #     'booking_time_day_cos',
    #     'booking_time_month_sin',
    #     'booking_time_month_cos',
    #     'booking_time_quarter_sin',
    #     'booking_time_quarter_cos',
    #     'booking_time_week_sin',
    #     'booking_time_week_cos',
    #     'booking_time_dow_sin',
    #     'booking_time_dow_cos',
    #     'booking_time_doy_sin',
    #     'booking_time_doy_cos',
    #     'booking_time_season_num_sin',
    #     'booking_time_season_num_cos',
    #     'holidays_during_booking',
]


prop_location_score_features = [
    'prop_location_score_diff',
    #     'prop_location_score_diff_7',
    'prop_location_score_sum',
    #     'prop_location_score_sum_7',
    'prop_location_score_mul',
    'prop_location_score_diff_log',
    'prop_location_score_sum_log',
    'prop_location_score_mul_log',
]


competitor_features = [
    'higher_price_than_expedia',
    #     'lower_price_than_expedia',
    #     'same_price_as_expedia',
    #     'comp_num_avail',
    'ratio_of_comp_with_higher_price_than_expedia',
    'ratio_of_comp_with_lower_price_than_expedia',
    #     'comp1_rate_percent_diff_signed',
    #     'comp2_rate_percent_diff_signed',
    #     'comp3_rate_percent_diff_signed',
    #     'comp4_rate_percent_diff_signed',
    'comp5_rate_percent_diff_signed',
    #     'comp6_rate_percent_diff_signed',
    #     'comp7_rate_percent_diff_signed',
    'comp8_rate_percent_diff_signed',

    #  'comp1_rate_percent_diff_log_2',
    #     'comp1_rate_percent_diff_log_10',
    #  'comp2_rate_percent_diff_log_2',
    #     'comp2_rate_percent_diff_log_10',
    #  'comp3_rate_percent_diff_log_2',
    #     'comp3_rate_percent_diff_log_10',
    #  'comp4_rate_percent_diff_log_2',
    #     'comp4_rate_percent_diff_log_10',
    #  'comp5_rate_percent_diff_log_2',
    #     'comp5_rate_percent_diff_log_10',
    #  'comp6_rate_percent_diff_log_2',
    #     'comp6_rate_percent_diff_log_10',
    #  'comp7_rate_percent_diff_log_2',
    #     'comp7_rate_percent_diff_log_10',
    #  'comp8_rate_percent_diff_log_2',
    #     'comp8_rate_percent_diff_log_10',
]

price_features = [
    'price_per_day',
    #     'price_per_trip',

    #     'price_usd_pow_0.33',
    #     'price_usd_pow_0.5',
    #     'price_usd_pow_2',
    #     'price_usd_log_2',
    'price_usd_log_10',
    #     'price_usd_log_2.72',
    #     'price_usd_reciprocal',

    'visitor_hist_adr_usd_pow_0.33',
    'visitor_hist_adr_usd_pow_0.5',
    'visitor_hist_adr_usd_pow_2',
    #     'visitor_hist_adr_usd_log_2',
    #     'visitor_hist_adr_usd_log_10',
    #     'visitor_hist_adr_usd_log_2.72',
    'visitor_hist_adr_usd_reciprocal',

    'prop_log_historical_price_pow_0.33',
    #     'prop_log_historical_price_pow_0.5',
    #     'prop_log_historical_price_pow_2',
    #     'prop_log_historical_price_log_2',
    'prop_log_historical_price_log_10',
    #     'prop_log_historical_price_log_2.72',
    'prop_log_historical_price_reciprocal',

    #     'price_per_trip_pow_0.33',
    #     'price_per_trip_pow_0.5',
    #     'price_per_trip_pow_2',
    #     'price_per_trip_log_2',
    #     'price_per_trip_log_10',
    #     'price_per_trip_log_2.72',
    #     'price_per_trip_reciprocal',

    'price_diff_to_usr_hist',
    'price_ratio_to_usr_hist',
    'price_diff_to_place_hist_price',
    #     'place_hist_price_diff_usr_hist',
]


star_features = ['prop_star_usr_hist_diff',
                 'prop_review_usr_hist_diff',
                 'prop_starrating_bonus_chain',
                 'prop_star_review_sum',
                 'prop_star_review_diff',
                 'prop_star_review_min',
                 'prop_star_review_max',
                 'prop_star_review_mean', ]

price_star_features = ['price_per_star',
                       'price_per_review_score',
                       #                        'usr_price_per_star',
                       #                        'price_per_star_diff_with_usr',
                       #                        'price_per_review_diff_with_usr',
                       'price_per_star_log_10',
                       'price_per_review_score_log_10',
                       'usr_price_per_star_log_10',
                       ]


trip_detail_features = [
    'travel_inside',
    #     'trans_ocean',
    'booking_weeks_ahead',
    #     'booking_months_ahead',
    'len_of_stay_to_booking_ahead_ratio',
    #     'len_of_stay_to_booking_ahead_diff',
    #     'srch_length_of_stay_in_weeks',
    #     'date',
    #     'trip_start_date',
    #     'trip_end_date',
    #     'holidays_during_trip',
    #     'business_trip',
    'children_per_adult',
    #     'total_people',
    'people_per_room',
    'separate_children_room'
]


aggregate_features = [
    'price_index_per_srch_id',
    'price_index_per_srch_destination_id',
    'price_index_per_prop_id',
    #     'mean_click_bool_per_prop_id',
    #     'mean_booking_bool_per_prop_id',
    #     'mean_srch_query_affinity_score',
    #     'mean_position_per_prop_id',
]

uncategorized_features = [
    #     'was_sold_in_that_period',
    'srch_query_affinity_score_prob'
]

##### AGGREGATION FEATURES #####
per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter = [
    #     'mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    #     'median_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    #     'min_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    #     'max_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_diff_to_mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_diff_to_median_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_median_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_diff_to_min_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_min_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    #     'price_per_day_diff_to_max_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_max_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter']


per_visitor_location_country_id_per_prop_id = [
    'mean_price_per_day_per_visitor_location_country_id_per_prop_id',
    'median_price_per_day_per_visitor_location_country_id_per_prop_id',
    #     'min_price_per_day_per_visitor_location_country_id_per_prop_id',
    'max_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_rel_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_id',
    'price_per_day_rel_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_id',

    #     'mean_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    #     'median_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    #     'min_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    #     'max_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    #     'price_per_day_rel_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter'
]

per_day_per_prop_id_per_trip_start_date_quarter = [
    'mean_price_per_day_per_prop_id_per_trip_start_date_quarter',
    #     'median_price_per_day_per_prop_id_per_trip_start_date_quarter',
    #     'min_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'max_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_mean_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_mean_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_median_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_median_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_min_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_min_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_max_price_per_day_per_prop_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_max_price_per_day_per_prop_id_per_trip_start_date_quarter'
]

per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter = [
    #     'mean_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    #     'median_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    #     'min_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    #     'max_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_diff_to_mean_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_mean_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_diff_to_median_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_median_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_diff_to_min_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_min_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_diff_to_max_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_max_price_per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter'
]

per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2 = [
    'mean_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    #     'median_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'min_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'max_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    #     'price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'price_per_day_rel_diff_to_min_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'price_per_day_diff_to_max_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2',
    'price_per_day_rel_diff_to_max_price_per_day_per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2'
]

price_per_day_per_visitor_location_country_id_per_prop_starrating = [
    'mean_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'median_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    #  'min_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    #  'max_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_rel_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_starrating',
    'price_per_day_rel_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_starrating'
]

per_srch_destination_id_per_srch_room_count_per_prop_review_score = [
    #     'mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    #  'median_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'min_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    #  'max_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_diff_to_mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_rel_diff_to_mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_diff_to_median_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_rel_diff_to_median_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_diff_to_min_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_rel_diff_to_min_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_diff_to_max_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score',
    'price_per_day_rel_diff_to_max_price_per_day_per_srch_destination_id_per_srch_room_count_per_prop_review_score'
]

per_day_per_srch_destination_id_per_trip_start_date_quarter = [
    #     'mean_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    #  'median_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    #  'min_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    #  'max_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_mean_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_mean_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_median_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_median_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_min_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    #  'price_per_day_rel_diff_to_min_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    'price_per_day_diff_to_max_price_per_day_per_srch_destination_id_per_trip_start_date_quarter',
    'price_per_day_rel_diff_to_max_price_per_day_per_srch_destination_id_per_trip_start_date_quarter'
]

per_day_per_prop_country_id = [
    #     'mean_price_per_day_per_prop_country_id',
    #  'median_price_per_day_per_prop_country_id',
    'min_price_per_day_per_prop_country_id',
    'max_price_per_day_per_prop_country_id',
    #  'price_per_day_diff_to_mean_price_per_day_per_prop_country_id',
    #  'price_per_day_rel_diff_to_mean_price_per_day_per_prop_country_id',
    'price_per_day_diff_to_median_price_per_day_per_prop_country_id',
    'price_per_day_rel_diff_to_median_price_per_day_per_prop_country_id',
    'price_per_day_diff_to_min_price_per_day_per_prop_country_id',
    'price_per_day_rel_diff_to_min_price_per_day_per_prop_country_id',
    'price_per_day_diff_to_max_price_per_day_per_prop_country_id',
    'price_per_day_rel_diff_to_max_price_per_day_per_prop_country_id'
]

per_visitor_location_country_id_per_prop_review_score = [
    'mean_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    'median_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    #  'min_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    #  'max_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    'price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    'price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    'price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    'price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    'price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    #  'price_per_day_rel_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    'price_per_day_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_review_score',
    #  'price_per_day_rel_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_review_score'
]

per_visitor_location_country_id_per_srch_destination_id = [
    'mean_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    #  'median_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    #  'min_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    #  'max_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_rel_diff_to_min_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_diff_to_max_price_per_day_per_visitor_location_country_id_per_srch_destination_id',
    'price_per_day_rel_diff_to_max_price_per_day_per_visitor_location_country_id_per_srch_destination_id'
]

per_prop_id = [
    'mean_price_per_day_per_prop_id',
    'median_price_per_day_per_prop_id',
    'min_price_per_day_per_prop_id',
    'max_price_per_day_per_prop_id',
    'price_per_day_diff_to_mean_price_per_day_per_prop_id',
    'price_per_day_rel_diff_to_mean_price_per_day_per_prop_id',
    'price_per_day_diff_to_median_price_per_day_per_prop_id',
    'price_per_day_rel_diff_to_median_price_per_day_per_prop_id',
    'price_per_day_diff_to_min_price_per_day_per_prop_id',
    'price_per_day_rel_diff_to_min_price_per_day_per_prop_id',
    'price_per_day_diff_to_max_price_per_day_per_prop_id',
    'price_per_day_rel_diff_to_max_price_per_day_per_prop_id'
]

per_day_per_srch_destination_id_per_prop_review_score = [
    #     'mean_price_per_day_per_srch_destination_id_per_prop_review_score',
    #  'median_price_per_day_per_srch_destination_id_per_prop_review_score',
    'min_price_per_day_per_srch_destination_id_per_prop_review_score',
    'max_price_per_day_per_srch_destination_id_per_prop_review_score',
    'price_per_day_diff_to_mean_price_per_day_per_srch_destination_id_per_prop_review_score',
    'price_per_day_rel_diff_to_mean_price_per_day_per_srch_destination_id_per_prop_review_score',
    'price_per_day_diff_to_median_price_per_day_per_srch_destination_id_per_prop_review_score',
    'price_per_day_rel_diff_to_median_price_per_day_per_srch_destination_id_per_prop_review_score',
    #  'price_per_day_diff_to_min_price_per_day_per_srch_destination_id_per_prop_review_score',
    'price_per_day_rel_diff_to_min_price_per_day_per_srch_destination_id_per_prop_review_score',
    'price_per_day_diff_to_max_price_per_day_per_srch_destination_id_per_prop_review_score',
    'price_per_day_rel_diff_to_max_price_per_day_per_srch_destination_id_per_prop_review_score'
]

cols_to_use = (original_cols + booking_date_features + prop_location_score_features +
               competitor_features + price_features +
               trip_detail_features + aggregate_features + uncategorized_features +
               star_features + price_star_features +

               per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter +
               per_visitor_location_country_id_per_prop_id +
               per_day_per_prop_id_per_trip_start_date_quarter +
               per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter +
               #                per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2 +
               price_per_day_per_visitor_location_country_id_per_prop_starrating +
               per_srch_destination_id_per_srch_room_count_per_prop_review_score +
               per_day_per_srch_destination_id_per_trip_start_date_quarter +
               per_day_per_prop_country_id +
               per_visitor_location_country_id_per_prop_review_score +
               per_visitor_location_country_id_per_srch_destination_id +
               per_prop_id +
               per_day_per_srch_destination_id_per_prop_review_score)

len(cols_to_use)

186

In [20]:
assert len(cols_to_use) == len(set(cols_to_use)), 'some columns are duplicated'

In [21]:
CAT_FEATURES = ['srch_id',
                'site_id',
                'visitor_location_country_id',
                'prop_country_id',
                'prop_id',
                'srch_destination_id',
                'week_id',
                'season_num', 'day', 'month', 'year', 'quarter', 'week', 'dow'
                ]

CAT_FEATURES = [c for c in CAT_FEATURES if c in cols_to_use]

# bool_cols = ['prop_brand_bool',
#              'promotion_flag',
#              'srch_saturday_night_bool',
#              'random_bool']

# drop_cols = ['usr_extra_pay']

group_col = 'srch_id'
predict_item_col = 'prop_id'

DROP_DATETIME_TYPE = True

# Read data

In [23]:
data_path = 'data' #'../input/expedia-with-aggregated-features-15-may' 

In [22]:
# # 15.05.2022 Oleg: for feature listing purposes. Do not used in modeling

# val_df = pd.read_feather(os.path.join(data_path, 'X_val.feather'))

# cols = val_df.columns.to_list()
# cols = [c for c in cols if 'per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter' not in c]
# cols = [c for c in cols if 'per_day_per_visitor_location_country_id_per_prop_id' not in c]
# cols = [c for c in cols if 'per_day_per_prop_id_per_trip_start_date_quarter' not in c]
# cols = [c for c in cols if 'per_day_per_prop_id_per_srch_room_count_per_trip_start_date_quarter' not in c]
# cols = [c for c in cols if 'per_visitor_location_country_id_per_srch_destination_id_per_prop_location_score2' not in c]
# cols = [c for c in cols if 'price_per_day_per_visitor_location_country_id_per_prop_starrating' not in c]
# cols = [c for c in cols if 'per_srch_destination_id_per_srch_room_count_per_prop_review_score' not in c]
# cols = [c for c in cols if not c.endswith('per_day_per_srch_destination_id_per_trip_start_date_quarter')]
# cols = [c for c in cols if not c.endswith('per_day_per_prop_country_id')]
# cols = [c for c in cols if not c.endswith('per_visitor_location_country_id_per_prop_review_score')]
# cols = [c for c in cols if not c.endswith('per_visitor_location_country_id_per_srch_destination_id')]
# cols = [c for c in cols if not c.endswith('per_prop_id')]
# cols = [c for c in cols if 'per_day_per_srch_destination_id_per_prop_review_score' not in c]
# print(len(cols))
# [c for c in cols if 'per_day_per_srch_destination_id_per_prop_review_score' in c]
# # [c for c in cols if c.endswith('per_prop_id')]
# # sorted([c for c in cols if c not in cols_to_use and 'per_srch_destination_id' in c][100:])



In [32]:
X_train = pd.read_feather(os.path.join(data_path, 'X_train.feather'), columns=cols_to_use)
y_train = pd.read_feather(os.path.join(data_path, 'y_train.feather'))['target']
print('X_train.shape', X_train.shape)
X_train.head()

X_train.shape (3969251, 186)


,srch_id,site_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_booking_window,srch_children_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp2_rate_percent_diff,comp3_rate_percent_diff,comp5_rate_percent_diff,date_time_ordinal,prop_location_score_diff,prop_location_score_sum,prop_location_score_mul,prop_location_score_diff_log,prop_location_score_sum_log,prop_location_score_mul_log,higher_price_than_expedia,ratio_of_comp_with_higher_price_than_expedia,ratio_of_comp_with_lower_price_than_expedia,comp5_rate_percent_diff_signed,comp8_rate_percent_diff_signed,price_per_day,price_usd_log_10,visitor_hist_adr_usd_pow_0.33,visitor_hist_adr_usd_pow_0.5,visitor_hist_adr_usd_pow_2,visitor_hist_adr_usd_reciprocal,prop_log_historical_price_pow_0.33,prop_log_historical_price_log_10,prop_log_historical_price_reciprocal,price_diff_to_usr_hist,price_ratio_to_usr_hist,price_diff_to_place_hist_price,travel_inside,booking_weeks_ahead,len_of_stay_to_booking_ahead_ratio,children_per_adult,people_per_room,separate_children_room,price_index_per_srch_id,price_index_per_srch_destination_id,price_index_per_prop_id,srch_query_affinity_score_prob,prop_star_usr_hist_diff,prop_review_usr_hist_diff,prop_starrating_bonus_chain,prop_star_review_sum,prop_star_review_diff,prop_star_review_min,prop_star_review_max,prop_star_review_mean,price_per_star,price_per_review_score,price_per_star_log_10,price_per_review_score_log_10,usr_price_per_star_log_10,price_per_day_diff_to_mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter,price_per_day_rel_diff_to_mean_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter,price_per_day_diff_to_median_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter,price_per_day_rel_diff_to_median_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter,price_per_day_diff_to_min_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter,price_per_day_rel_diff_to_min_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter,price_per_day_rel_diff_to_max_price_per_day_per_srch_destination_id_per_srch_room_count_per_trip_start_date_quarter,mean_price_per_day_per_visitor_location_country_id_per_prop_id,median_price_per_day_per_visitor_location_country_id_per_prop_id,max_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_rel_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_rel_diff_to_max_price_per_day_per_visitor_location_country_id_per_prop_id,price_per_day_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter,price_per_day_rel_diff_to_mean_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter,price_per_day_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter,price_per_day_rel_diff_to_median_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter,price_per_day_diff_to_min_price_per_day_per_visitor_location_country_id_per_prop_id_per_trip_start_date_quarter,price_per_day_diff_to_max_price_per_day_per_visitor_

In [34]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [35]:
X_val = pd.read_feather(os.path.join(data_path, 'X_val.feather'), columns=cols_to_use)
y_val = pd.read_feather(os.path.join(data_path, 'y_val.feather'))['target']    
print('X_val.shape', X_val.shape)

X_val.shape (496664, 186)


In [38]:
submission_df = pd.read_feather(os.path.join(data_path, 'submission_df_preprocessed.feather'), columns=cols_to_use)
print('submission_df Shape', submission_df.shape)

submission_df Shape (4959183, 186)


# Fit final model

In [45]:
def handle_time(df):
    end = df.trip_end_date.astype(int) // 10**9
    start = df.trip_start_date.astype(int) // 10**9
    
    df.drop(['trip_end_date', 'trip_start_date'], axis=1, inplace=True)
    df['trip_length_seconds'] = end - start

# drop_cols = ['date_time', 'date']
# X_train.drop(drop_cols, axis=1, inplace=True)
# X_val.drop(drop_cols, axis=1, inplace=True)
# submission_df.drop(drop_cols, axis=1, inplace=True)

# handle_time(X_train)
# handle_time(X_val)
# handle_time(submission_df)
qids_train = X_train.groupby('srch_id')['srch_id'].count().to_numpy()
qids_val = X_val.groupby('srch_id')['srch_id'].count().to_numpy()

In [57]:
clf = LGBMRanker(
        objective="lambdarank",
        metric="ndcg",
        n_estimators=5000,
        learning_rate=0.025,
        random_state=42,
        seed=42,
        boosting="dart",
    )

In [58]:
%%time
clf.fit(
    X = X_train,
    y = y_train,
    group = qids_train,
    eval_set=[(X_val, y_val)],
    eval_group=[qids_val],
    verbose=10,
)

/Users/nedim.azar/Library/Python/3.8/lib/python/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] seed is set=42, random_state=42 will be ignored. Current value: seed=42
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[10]	valid_0's ndcg@1: 0.189043	valid_0's ndcg@2: 0.263532	valid_0's ndcg@3: 0.3072	valid_0's ndcg@4: 0.339719	valid_0's ndcg@5: 0.364255
[20]	valid_0's ndcg@1: 0.189412	valid_0's ndcg@2: 0.264851	valid_0's ndcg@3: 0.310046	valid_0's ndcg@4: 0.341598	valid_0's ndcg@5: 0.365579
[30]	valid_0's ndcg@1: 0.188222	valid_0's ndcg@2: 0.264542	valid_0's ndcg@3: 0.310907	valid_0's ndcg@4: 0.343194	valid_0's ndcg@5: 0.36714
[40]	valid_0's ndcg@1: 0.190125	valid_0's ndcg@2: 0.265165	valid_0's ndcg@3: 0.312177	valid_0's ndcg@4: 0.344723	valid_0's ndcg@5: 0.3686
[50]	valid_0's ndcg@1: 0.18997	valid_0's ndcg@2: 0.265705	valid_0's ndcg@3: 0.312153	valid_0's ndcg@4: 0.344477	valid_0's ndcg@5: 0.36881
[60]	valid_0's ndcg@1: 0.191065	valid_0's ndcg@2: 0.266597	valid_0's ndcg@3: 0.313416	valid_0's ndcg@4: 0.

[580]	valid_0's ndcg@1: 0.197455	valid_0's ndcg@2: 0.273516	valid_0's ndcg@3: 0.319758	valid_0's ndcg@4: 0.352292	valid_0's ndcg@5: 0.376772
[590]	valid_0's ndcg@1: 0.198052	valid_0's ndcg@2: 0.274313	valid_0's ndcg@3: 0.319725	valid_0's ndcg@4: 0.352666	valid_0's ndcg@5: 0.376873
[600]	valid_0's ndcg@1: 0.198646	valid_0's ndcg@2: 0.27459	valid_0's ndcg@3: 0.319998	valid_0's ndcg@4: 0.353324	valid_0's ndcg@5: 0.377147
[610]	valid_0's ndcg@1: 0.198749	valid_0's ndcg@2: 0.274596	valid_0's ndcg@3: 0.320371	valid_0's ndcg@4: 0.352926	valid_0's ndcg@5: 0.377307
[620]	valid_0's ndcg@1: 0.19835	valid_0's ndcg@2: 0.274005	valid_0's ndcg@3: 0.320277	valid_0's ndcg@4: 0.353211	valid_0's ndcg@5: 0.377394
[630]	valid_0's ndcg@1: 0.198102	valid_0's ndcg@2: 0.273505	valid_0's ndcg@3: 0.319966	valid_0's ndcg@4: 0.352767	valid_0's ndcg@5: 0.37727
[640]	valid_0's ndcg@1: 0.198552	valid_0's ndcg@2: 0.273745	valid_0's ndcg@3: 0.320377	valid_0's ndcg@4: 0.353274	valid_0's ndcg@5: 0.377459
[650]	valid_0's 

[1170]	valid_0's ndcg@1: 0.201076	valid_0's ndcg@2: 0.277334	valid_0's ndcg@3: 0.324794	valid_0's ndcg@4: 0.357383	valid_0's ndcg@5: 0.381626
[1180]	valid_0's ndcg@1: 0.201272	valid_0's ndcg@2: 0.277639	valid_0's ndcg@3: 0.324895	valid_0's ndcg@4: 0.357551	valid_0's ndcg@5: 0.381668
[1190]	valid_0's ndcg@1: 0.201272	valid_0's ndcg@2: 0.277283	valid_0's ndcg@3: 0.324824	valid_0's ndcg@4: 0.357428	valid_0's ndcg@5: 0.381608
[1200]	valid_0's ndcg@1: 0.201223	valid_0's ndcg@2: 0.277296	valid_0's ndcg@3: 0.324752	valid_0's ndcg@4: 0.357386	valid_0's ndcg@5: 0.381699
[1210]	valid_0's ndcg@1: 0.201572	valid_0's ndcg@2: 0.277635	valid_0's ndcg@3: 0.325166	valid_0's ndcg@4: 0.357964	valid_0's ndcg@5: 0.381926
[1220]	valid_0's ndcg@1: 0.201274	valid_0's ndcg@2: 0.277044	valid_0's ndcg@3: 0.324993	valid_0's ndcg@4: 0.357765	valid_0's ndcg@5: 0.381823
[1230]	valid_0's ndcg@1: 0.201223	valid_0's ndcg@2: 0.27669	valid_0's ndcg@3: 0.324946	valid_0's ndcg@4: 0.357817	valid_0's ndcg@5: 0.381764
[1240]	

[1750]	valid_0's ndcg@1: 0.203579	valid_0's ndcg@2: 0.279929	valid_0's ndcg@3: 0.327423	valid_0's ndcg@4: 0.36019	valid_0's ndcg@5: 0.384742
[1760]	valid_0's ndcg@1: 0.203627	valid_0's ndcg@2: 0.28008	valid_0's ndcg@3: 0.32746	valid_0's ndcg@4: 0.360316	valid_0's ndcg@5: 0.384941
[1770]	valid_0's ndcg@1: 0.203428	valid_0's ndcg@2: 0.279935	valid_0's ndcg@3: 0.327489	valid_0's ndcg@4: 0.360282	valid_0's ndcg@5: 0.38475
[1780]	valid_0's ndcg@1: 0.20373	valid_0's ndcg@2: 0.280197	valid_0's ndcg@3: 0.327672	valid_0's ndcg@4: 0.360297	valid_0's ndcg@5: 0.38506
[1790]	valid_0's ndcg@1: 0.204181	valid_0's ndcg@2: 0.280087	valid_0's ndcg@3: 0.327771	valid_0's ndcg@4: 0.360225	valid_0's ndcg@5: 0.385188
[1800]	valid_0's ndcg@1: 0.203933	valid_0's ndcg@2: 0.279901	valid_0's ndcg@3: 0.327339	valid_0's ndcg@4: 0.359805	valid_0's ndcg@5: 0.384908
[1810]	valid_0's ndcg@1: 0.203632	valid_0's ndcg@2: 0.279985	valid_0's ndcg@3: 0.327617	valid_0's ndcg@4: 0.360209	valid_0's ndcg@5: 0.385003
[1820]	valid

...

[2910]	valid_0's ndcg@1: 0.206025	valid_0's ndcg@2: 0.282554	valid_0's ndcg@3: 0.33003	valid_0's ndcg@4: 0.362994	valid_0's ndcg@5: 0.388144
[2920]	valid_0's ndcg@1: 0.206125	valid_0's ndcg@2: 0.282565	valid_0's ndcg@3: 0.330088	valid_0's ndcg@4: 0.363131	valid_0's ndcg@5: 0.388196
[2930]	valid_0's ndcg@1: 0.206278	valid_0's ndcg@2: 0.282822	valid_0's ndcg@3: 0.330097	valid_0's ndcg@4: 0.36327	valid_0's ndcg@5: 0.388272
[2940]	valid_0's ndcg@1: 0.206077	valid_0's ndcg@2: 0.282726	valid_0's ndcg@3: 0.330104	valid_0's ndcg@4: 0.363258	valid_0's ndcg@5: 0.388217
[2950]	valid_0's ndcg@1: 0.206177	valid_0's ndcg@2: 0.282932	valid_0's ndcg@3: 0.330327	valid_0's ndcg@4: 0.363429	valid_0's ndcg@5: 0.388327
[2960]	valid_0's ndcg@1: 0.206127	valid_0's ndcg@2: 0.282881	valid_0's ndcg@3: 0.330277	valid_0's ndcg@4: 0.36339	valid_0's ndcg@5: 0.38812
[2970]	valid_0's ndcg@1: 0.205926	valid_0's ndcg@2: 0.28287	valid_0's ndcg@3: 0.330024	valid_0's ndcg@4: 0.363208	valid_0's ndcg@5: 0.388248
[2980]	vali

[3490]	valid_0's ndcg@1: 0.208029	valid_0's ndcg@2: 0.284126	valid_0's ndcg@3: 0.331828	valid_0's ndcg@4: 0.36434	valid_0's ndcg@5: 0.389702
[3500]	valid_0's ndcg@1: 0.207881	valid_0's ndcg@2: 0.284174	valid_0's ndcg@3: 0.332115	valid_0's ndcg@4: 0.364452	valid_0's ndcg@5: 0.389759
[3510]	valid_0's ndcg@1: 0.208081	valid_0's ndcg@2: 0.284519	valid_0's ndcg@3: 0.332128	valid_0's ndcg@4: 0.364489	valid_0's ndcg@5: 0.390008
[3520]	valid_0's ndcg@1: 0.207733	valid_0's ndcg@2: 0.284245	valid_0's ndcg@3: 0.331932	valid_0's ndcg@4: 0.364249	valid_0's ndcg@5: 0.389746
[3530]	valid_0's ndcg@1: 0.207781	valid_0's ndcg@2: 0.284341	valid_0's ndcg@3: 0.331662	valid_0's ndcg@4: 0.364314	valid_0's ndcg@5: 0.389646
[3540]	valid_0's ndcg@1: 0.207983	valid_0's ndcg@2: 0.284125	valid_0's ndcg@3: 0.331752	valid_0's ndcg@4: 0.364491	valid_0's ndcg@5: 0.389825
[3550]	valid_0's ndcg@1: 0.207733	valid_0's ndcg@2: 0.28411	valid_0's ndcg@3: 0.331629	valid_0's ndcg@4: 0.364436	valid_0's ndcg@5: 0.389808
[3560]	v

[4070]	valid_0's ndcg@1: 0.207835	valid_0's ndcg@2: 0.283317	valid_0's ndcg@3: 0.332109	valid_0's ndcg@4: 0.364485	valid_0's ndcg@5: 0.389674
[4080]	valid_0's ndcg@1: 0.208081	valid_0's ndcg@2: 0.283432	valid_0's ndcg@3: 0.332184	valid_0's ndcg@4: 0.364809	valid_0's ndcg@5: 0.390045
[4090]	valid_0's ndcg@1: 0.20823	valid_0's ndcg@2: 0.283474	valid_0's ndcg@3: 0.332161	valid_0's ndcg@4: 0.364958	valid_0's ndcg@5: 0.390025
[4100]	valid_0's ndcg@1: 0.208879	valid_0's ndcg@2: 0.283966	valid_0's ndcg@3: 0.332712	valid_0's ndcg@4: 0.365197	valid_0's ndcg@5: 0.390304
[4110]	valid_0's ndcg@1: 0.208131	valid_0's ndcg@2: 0.283619	valid_0's ndcg@3: 0.332254	valid_0's ndcg@4: 0.364865	valid_0's ndcg@5: 0.389998
[4120]	valid_0's ndcg@1: 0.208129	valid_0's ndcg@2: 0.283568	valid_0's ndcg@3: 0.332498	valid_0's ndcg@4: 0.364708	valid_0's ndcg@5: 0.390207
[4130]	valid_0's ndcg@1: 0.208729	valid_0's ndcg@2: 0.283879	valid_0's ndcg@3: 0.332644	valid_0's ndcg@4: 0.365038	valid_0's ndcg@5: 0.3905
[4140]	va

[4650]	valid_0's ndcg@1: 0.207725	valid_0's ndcg@2: 0.284093	valid_0's ndcg@3: 0.332535	valid_0's ndcg@4: 0.365071	valid_0's ndcg@5: 0.390357
[4660]	valid_0's ndcg@1: 0.207775	valid_0's ndcg@2: 0.284199	valid_0's ndcg@3: 0.332501	valid_0's ndcg@4: 0.365005	valid_0's ndcg@5: 0.390262
[4670]	valid_0's ndcg@1: 0.207826	valid_0's ndcg@2: 0.284446	valid_0's ndcg@3: 0.332504	valid_0's ndcg@4: 0.365073	valid_0's ndcg@5: 0.390347
[4680]	valid_0's ndcg@1: 0.207278	valid_0's ndcg@2: 0.284117	valid_0's ndcg@3: 0.332417	valid_0's ndcg@4: 0.364716	valid_0's ndcg@5: 0.390045
[4690]	valid_0's ndcg@1: 0.207624	valid_0's ndcg@2: 0.284251	valid_0's ndcg@3: 0.332676	valid_0's ndcg@4: 0.36499	valid_0's ndcg@5: 0.390355
[4700]	valid_0's ndcg@1: 0.207175	valid_0's ndcg@2: 0.284031	valid_0's ndcg@3: 0.332465	valid_0's ndcg@4: 0.364899	valid_0's ndcg@5: 0.390326
[4710]	valid_0's ndcg@1: 0.207424	valid_0's ndcg@2: 0.284039	valid_0's ndcg@3: 0.332602	valid_0's ndcg@4: 0.365079	valid_0's ndcg@5: 0.390215
[4720]	

LGBMRanker(boosting='dart', learning_rate=0.025, metric='ndcg',
           n_estimators=5000, objective='lambdarank', random_state=42, seed=42)

In [59]:
predictions = clf.predict(submission_df)

In [60]:
submission_file = submission_df[['srch_id', 'prop_id']]

In [65]:
submission_file.to_csv('big_submission_scores.csv', index=False)

In [61]:
submission_file['ord'] = predictions

/var/folders/sg/n9mvg73s2cldzl8k88kctzgh0000gn/T/ipykernel_22288/608820714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_file['ord'] = predictions


In [62]:
submission_file.sort_values(by=['srch_id', 'ord'], ascending=[True, False], inplace=True)

/var/folders/sg/n9mvg73s2cldzl8k88kctzgh0000gn/T/ipykernel_22288/4266963365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_file.sort_values(by=['srch_id', 'ord'], ascending=[True, False], inplace=True)


In [63]:
submission_file.drop(['ord'], axis=1).to_csv('submission_big_clf.csv', index=False)

# Feature importance

In [ ]:
# del test_pool; gc.collect()

X_val = pd.read_feather(os.path.join(data_path, 'X_val.feather'), columns=cols_to_use).sample(100000)
X_val = X_val[X_val.index.isin((X_val[group_col].value_counts() > 1).index)]
X_val.sort_values(group_col, inplace=True)
y_val = pd.read_feather(os.path.join(data_path, 'y_val.feather')).loc[X_val.index, 'target']
print('X_val.shape', X_val.shape)

val_pool = Pool(data=X_val,
                label=y_val,
                group_id=X_val[group_col],
                cat_features=CAT_FEATURES,
                )

In [ ]:
g = model.plot_tree(
    tree_idx=0,
    pool=val_pool
)
g

In [ ]:
# g.save('tree.pdf')

In [ ]:
# from feature_importance import get_and_plot_feature_imp_catboost

# COLLECT_N_IMPORTANT_FEATURES = 150

# feature_score_SHAP, shap_values_raw = get_and_plot_feature_imp_catboost(model, X_val, y_val,
#                                                                         method='SHAP',
#                                                                         cat_features=CAT_FEATURES,
#                                                                         #                                                        plot_type='bar',
#                                                                         top_n=COLLECT_N_IMPORTANT_FEATURES)

# # important_features_collected.update(feature_score_SHAP.sort_values(
# #     ['Score'], ascending=False).values[:COLLECT_N_IMPORTANT_FEATURES, 0])

In [ ]:
# pd.set_option('max_rows', None)
# feature_score_SHAP.sort_values('Score', ascending=False)

# shap_values_raw

features each contributing to push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue.

In [ ]:
import shap
shap.initjs()

explainer = shap.Explainer(model)
shap_values = explainer(val_pool) # X_val

We can also just take the mean absolute value of the SHAP values for each feature to get a standard bar plot (produces stacked bars for multi-class outputs):

In [ ]:
pd.set_option('max_rows', None)
pd.set_option('display.max_colwidth', None)

import numpy as np
features = X_val.columns
mean_shaps = np.abs(shap_values.values).mean(0)
# mask = mean_shaps.argsort()[::-1]
# features[mask]
shaps_df = pd.DataFrame({'feature': features, 'shap': mean_shaps})
shaps_df[shaps_df['feature'].isin(cols_to_use)].sort_values('shap', ascending=True)

In [ ]:
shaps_df['per_col'] = shaps_df['feature'].apply(lambda x: ''.join(x.split('per_')[1:]))
shaps_df[shaps_df['per_col'] != ''].sort_values(['per_col', 'shap']).drop(['per_col'], axis=1)

In [ ]:
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.waterfall(shap_values[1])

In [ ]:
# visualize the first prediction's explanation with a force plot
shap.plots.force(shap_values[0])

In [ ]:
shap.plots.force(shap_values[10])

In [ ]:
# visualize all the training set predictions
shap.plots.force(shap_values[:100])

To understand how a single feature effects the output of the model we can plot the SHAP value of that feature vs. the value of the feature for all the examples in a dataset. Since SHAP values represent a feature's responsibility for a change in the model output. Vertical dispersion at a single value represents interaction effects with other features. To help reveal these interactions we can color by another feature. If we pass the whole explanation tensor to the color argument the scatter plot will pick the best feature to color by. 

In [ ]:
# create a dependence scatter plot to show the effect of a single feature across the whole dataset
shap.plots.scatter(shap_values[:100, "prop_location_score2"], color=shap_values[:100])

To get an overview of which features are most important for a model we can plot the SHAP values of every feature for every sample. The plot below sorts features by the sum of SHAP value magnitudes over all samples, and uses SHAP values to show the distribution of the impacts each feature has on the model output. The color represents the feature value (red high, blue low).

In [ ]:
# summarize the effects of all the features
shap.plots.beeswarm(shap_values, max_display=100)

In [ ]:
shap.plots.bar(shap_values, max_display=100)

SHAP interaction values are a generalization of SHAP values to higher order interactions. Fast exact computation of pairwise interactions are implemented for tree models with shap.TreeExplainer(model).shap_interaction_values(X). This returns a matrix for every prediction, where the main effects are on the diagonal and the interaction effects are off-diagonal. These values often reveal interesting hidden relationships

https://slundberg.github.io/shap/notebooks/plots/dependence_plot.html

In [ ]:
# explainer.shap_interaction_values(X_val.head(10))

In [ ]:
# feature_score_PredictionValuesChange = get_and_plot_feature_imp_catboost(
#         model, X_val, y_val,
#         cat_features=CAT_FEATURES,
#         method='PredictionValuesChange',
#         top_n=COLLECT_N_IMPORTANT_FEATURES)

#     important_features_collected.update(feature_score_PredictionValuesChange.sort_values(['Score'], ascending=False).values[:COLLECT_N_IMPORTANT_FEATURES, 0])

For each feature, LossFunctionChange represents the difference between the loss value of the model with this feature and without it.

+ more accurate than Prediction Values Change
+ more fast than Shap Values
- assumes that loss function changes for several features can be summed up

In [ ]:
import matplotlib.pyplot as plt

for method in ['LossFunctionChange']:
    fi = model.get_feature_importance(val_pool, type=method)
    feature_score_raw = pd.DataFrame(list(zip(X_val.columns, fi)),
                                     columns=['Feature', 'Score'])

    feature_score = feature_score_raw.sort_values(
        by='Score', ascending=False)

    plt.rcParams["figure.figsize"] = (7, len(feature_score) / 4)
    ax = feature_score.plot('Feature', 'Score', kind='barh', color='c')
    ax.set_title("Feature Importance using {}".format(method), fontsize=14)
    ax.set_xlabel("features")
    ax.invert_yaxis()
    plt.show()

In [ ]:
feature_score.sort_values('Feature')

In [ ]:
feature_score['per_col'] = feature_score['Feature'].apply(lambda x: ''.join(x.split('per_')[1:]))
feature_score[feature_score['per_col'] != ''].sort_values('per_col').drop(['per_col'], axis=1)